## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50
from resnet_builder import resnet
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
from keras.applications import NASNetLarge, MobileNetV2, ResNet101
from keras.models import Model
import os
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"

Using TensorFlow backend.


In [2]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
# 影像大小
IMAGE_SIZE = (32, 32)

# 影像類別數
NUM_CLASSES = 10

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 64

# 凍結網路層數
FREEZE_LAYERS = 0

# Epoch 數
NUM_EPOCHS = 50

# 模型輸出儲存的檔案
WEIGHTS_FINAL = 'model-NASNetLarge-final_2.h5'

In [4]:
train_datagen = ImageDataGenerator(width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   channel_shift_range=2,
                                   horizontal_flip=False,
                                   #samplewise_center=True,
                                   #samplewise_std_normalization=True,
                                   fill_mode='nearest')
"""
train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='lanczos',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)
"""
train_batches = train_datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)


In [5]:
# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet101(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
model = Model(inputs=net.input, outputs=output_layer)
for layer in model.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in model.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# load weights
model.load_weights('./HW_100_ResNet101.hdf5')
    
# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
model.compile(#optimizer=RMSprop(lr=1e-2), #Adam(lr=1e-2),
              optimizer=Adam(lr=1e-2),  
              loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [7]:
batch_size = BATCH_SIZE # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 150 # 訓練整個資料集共 30個循環

# Learning rate func
learning_rate_function = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=1e-8)
# set check point
file_path="./HW_100_ResNet101.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


# 訓練模型
"""
history = model.fit_generator(train_batches,
                                steps_per_epoch = len(train_batches) // BATCH_SIZE,
                                validation_data = (x_test, y_test),
                                epochs = NUM_EPOCHS,
                                callbacks=[learning_rate_function, checkpoint])
#"""
#"""
history = model.fit(x_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[learning_rate_function, checkpoint])
#"""

Train on 50000 samples, validate on 10000 samples
Epoch 1/150
50000/50000 [==============================] - 140s 3ms/step - loss: 4.7281 - accuracy: 0.1060 - val_loss: 7.8590 - val_accuracy: 0.1061

Epoch 00001: val_accuracy improved from -inf to 0.10610, saving model to ./HW_100_ResNet101.hdf5
Epoch 2/150
50000/50000 [==============================] - 137s 3ms/step - loss: 3.3714 - accuracy: 0.1359 - val_loss: 4.8306 - val_accuracy: 0.1474

Epoch 00002: val_accuracy improved from 0.10610 to 0.14740, saving model to ./HW_100_ResNet101.hdf5
Epoch 3/150
50000/50000 [==============================] - 137s 3ms/step - loss: 2.0822 - accuracy: 0.2013 - val_loss: 2.0088 - val_accuracy: 0.2345

Epoch 00003: val_accuracy improved from 0.14740 to 0.23450, saving model to ./HW_100_ResNet101.hdf5
Epoch 4/150
50000/50000 [==============================] - 137s 3ms/step - loss: 1.9855 - accuracy: 0.2434 - val_loss: 1.8582 - val_accuracy: 0.2851

Epoch 00004: val_accuracy improved from 0.23450 to 0.

In [6]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.9901716101646425
Test accuracy: 0.7422000169754028
